In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from os.path import join as pjoin
from scipy.io import savemat, loadmat

proj_dir = '/nfs/t3/workingshop/chenxiayu/study/FFA_pattern'
anal_dir = pjoin(proj_dir, 'analysis/s2/1080_fROI/refined_with_Kevin')
work_dir = '/nfs/s2/userhome/chenxiayu/workingdir/test/ffa_fig'

In [ ]:
meas_name = 'myelin'  # myelin, thickness
gids = [0, 1, 2]
hemis = ['lh', 'rh']
rois = ['pFus-face', 'mFus-face']
src_file = pjoin(anal_dir, 'structure/rois_v3_{hemi}_{meas}.pkl')
gid_file = pjoin(anal_dir, 'grouping/group_id_{hemi}.npy')
out_file = pjoin(work_dir, f'{meas_name}.mat')

hemi2data = {}
hemi2gid_vec = {}
for hemi in hemis:
    hemi2data[hemi] = pkl.load(open(src_file.format(hemi=hemi, meas=meas_name), 'rb'))
    hemi2gid_vec[hemi] = np.load(gid_file.format(hemi=hemi))

out_data = {}
for gid in gids:
    for roi in rois:
        for hemi in hemis:
            k = f'{hemi}_{roi}_{gid}'
            data = hemi2data[hemi]
            gid_vec = hemi2gid_vec[hemi]
            v = np.ones_like(gid_vec, np.float64) * np.nan
            gid_idx_vec = gid_vec == gid
            roi_idx = data['roi'].index(roi)
            v[gid_idx_vec] = data['meas'][roi_idx][gid_idx_vec]

savemat(out_file, out_data)